<a href="https://colab.research.google.com/github/davidnoone/PHYS332_FluidExamples/blob/main/PotentialObstacle_Direct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Potential flow around an obsticle

HERE FORMULATED BY VELOCITY POTENTIAL

We wish to solve the potential flow problem for flow around an obsticle. 
The problem can be cast as a possion equation, such that

$$
\nabla^2 \phi = 0
$$

where $\phi$ is the velocity potential. For a rectanglar 2d domain, the background flow is constant in the "x" direction with speed U. This sets the inflow and outflow conditions to be linearly increasing $\psi$ from the lower (y=0) to upper (y = Lmax) boundaries. We assume the flow is undisturbed (u=U) on the upper and lower boundaries.

The boundary condition on the surface of the obsticle is that there is no perpendicular velocity. That is, the streamfunction is constant along the obsticle surface, or the gradient of the velocity potential is zero. 

With these two conditions the problem is solvable.

## Numerical approach
We can approximate the Laplacian unsing finite differeces on a computational grid with spacing $\Delta x$, $\Delta y$, such that

$$
\nabla^2 \phi \approx
              \frac{\phi_{i-1,j} - 2 \phi_{i,j} +\phi_{i+1,j}}{\Delta x^2} 
            + \frac{\phi_{i,j-1} - 2 \phi_{i,j} +\phi_{i,j+1}}{\Delta y^2} = b
$$

Where b on the right is a constraint (i.e., divergence). 

An itterative solver can be used, and easily constructed. However, for small problems one can find a direct solution writing the form:


$$
a_x \phi_{i-1,j} + a_x \phi_{i+1,j} + 
a_y \phi_{i,j-1} + a_y \phi_{i,j+1} -
2(a_x + a_y) \phi_{i,j} = b_{i,j}
$$

Which takes the matrix form:
$$
A X = B
$$

The solution found by inverting A to find the matrix $X$ composed of values of $\phi$. A is large: a square matrix of size $(nx \times ny)\times (nx \times ny)$. And, fortunatly, it may be compactly stored since in general it is early diagonal with 5 diagonal rows in the sparse matrix. The python "sparce" module can manage this. 

Notce that in the case of obsticals within the flow the condition of free slip condition (i.e., no normal vector flow) means the elements need to be adjusted such that the gradient of $\phi$ vanishes in the normal direction. 


## Finalizing
With the velocity potential known known, the velocity components can be computed from the gradient. And with the velocites known, the pressure field can be computed from the Bernoilli principle:
$$
p = p_0 + \frac{1}{2} \rho (u^2 + v^2) 
$$



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
import time


# Obstacle

Let us define some obstacles. Here, a class, which can be used to make simple shapes: circle, squares, half circles,....
An "obstacle" is defined by a series of points that are considered to be a solid object. imask, jmask gives these points. One might wish to have several obstacles, and they are stored in a list. Later, we will wish just to know if a computational cell is in or not in an obsticale, so a function to create a binay/boolean mask from the i and j points will be used.

As written, obstacles can only be on the inner domain. There is no real limitation on this, just the code needs to allow for it.

In [ ]:


class Obstacle:
    def __init__(self, jmask, imask):
        self.jmask = jmask
        self.imask = imask

    @classmethod
    def as_shape(obj, X,Y,x0=0,y0=0,radius=0,shape='None'):
        """ define an obsticle of various tstape
            circle: set center and radius: x0, y0, radius
            half-circle: set center and radius: x0, y0, radius < 0  left, radius > 0 right
            semi-circle: set center and radius: x0, y0, radius < 0  top, radius > 0 bottom
            square: set center and radius: x0, y0, radius
            rectangle: set corners x0,y0, x1, y1
        """
        match shape:
          case "square":
            bmask = np.logical_and(np.abs(X-x0) <= radius , np.abs(Y-y0) <= radius)
          case "diamond":
            bmask = np.abs(X-x0) + np.abs(Y-y0) <= radius
          case "circle":
            bmask = (X-x0)**2 + (Y-y0)**2 <= radius*radius
          case "half-circle":
            bmask = (X-x0)**2 + (Y-y0)**2 <= radius*radius
            if (radius > 0):
                bmask = np.logical_and(bmask, X >= x0)
            else:
                bmask = np.logical_and(bmask, X <= x0)
          case "semi-circle":
            bmask = (X-x0)**2 + (Y-y0)**2 <= radius*radius
            if (radius > 0):
                bmask = np.logical_and(bmask, Y >= y0)
            else:
                bmask = np.logical_and(bmask, Y <= y0)
          case _:
            raise Exception("Unknown obsticle type")

        jmask, imask = np.where(bmask)

        return obj(jmask, imask) 


def make_bmask(X,Y, obs_list):
    """ Constructs a binary/boolean mask for the obstical(s) """
    bmask = np.zeros_like(X,dtype='bool')
    for obs in obs_list:
        bmask[obs.jmask, obs.imask] = True
    return bmask



# Sparse poisson solver

The scipy sparce library uses a sparce packing (from LAPACK) for sparce matrices. The "spsolve" function is ultimately called to do this. Setting up the matrix A  in the compressed sparce row (CSR) format is the challenge. This is most easily done using the "list of lists" format (via lil_matrix()), then letting pyhon do the conversion. lil_matrix can be slow, but super convenient for setting up the elemets by indexing in an intuitive way.

Notice that if kept square A can be quite large $(nx \times ny)^2$, and probably more memory than the machine has.  Each timension of A has 1d indices that are related to the original i and j points in the (ny,nx) domain. Thats is:

$$
k = j\times nx + i
$$

Thus the matrix A[kr,kc] one can set elements for the coefficients of the laplacing matrix for each row kr, relatve to the dependence on way in which it depends on other elemnets on the grid, captured by the columns, kc.

Most of the comutational time for this code is in building the A matrix. The solver is faster.

In [ ]:
def solve_poisson_direct(dy, dx, phi_free, bmask, b, nbnd=2):
    """ 
        Direct solver for 2d Poisson equarion 
         (the special case b=0 is the Laplace equation)

        optionally set width of boundary - essentially allowing neat plotting
        nbnd = 1    mathematically correct, and best
        nbnd = 2    good if using one sided derivatives for winds at edges
        nbnd = 3    good if using np.gradient
        Only applies to domain edges. Not to obsticles.

        Allows dx /= dy, but constant.

    """
    # Define the grid dimensions
    ny, nx = b.shape

    k1d = lambda j, i : j*nx + i

    ax = 1./(dx*dx)
    ay = 1./(dy*dy)

    # Set up the boundary conditions as freestream
    phi = np.zeros((ny, nx))
    for j in range(ny):
        phi[j,:] = phi_free

    # Flatten phi into a 1D array for the right-hand side vector
    brhs = b.copy().flatten()

    # Create the coefficient matrix A as a lil_matrix
    t_0 = time.time()
    A = sp.lil_matrix((ny*nx, ny*nx))

    # Edge boundaries
    for j in range(ny):
        for i in range(nx):
            k = j*nx + i
            #k = k1d(j,i)
            if j < nbnd or j >= ny-nbnd or i < nbnd or i >= nx-nbnd:
                A[k, k] = 1.0         # Boundary condition, no off elements
                brhs[k] = phi[j, i]   # Boundary value


    # Inner domain: inside boundarys
    for j in range(nbnd,ny-nbnd):
        for i in range(nbnd,nx-nbnd):
            k = j*nx + i

            # Check for mask conditions
            if bmask[j,i]:
                A[k, k ] = 1.0      # "1" means "assign b"
                brhs[k] = np.nan    # set this for neat plotting
                #brhs[k] = 0.

            else:
                A[k, k] = -2*(ax + ay)        # Interior point
                A[k, k-1 ] = ax    # Left neighbor (j, i-1)
                A[k, k+1 ] = ax    # Right neighbor (j, i+1)
                A[k, k-nx] = ay    # Upper neighbor (j-1, i)
                A[k, k+nx] = ay    # Lower neighbor (j+1, i)

                # if mask to the side, set zero gradient that direction
                # (Could set a know gradient here to give a source)
                if bmask[j,i+1]:
                    A[k, k    ] += ax
                    A[k, k+1  ]  = 0.0
                if bmask[j,i-1]:
                    A[k, k    ] += ax
                    A[k, k-1  ]  = 0.0
                if bmask[j-1,i]:
                    A[k, k    ] += ay
                    A[k, k-nx ]  = 0.0
                if bmask[j+1,i]:
                    A[k, k    ] += ay
                    A[k, k+nx ]  = 0.0

    t_1 = time.time() - t_0

    # Solve the linear system using a sparse direct solver
    t_0 = time.time()
    x = sp.linalg.spsolve(A.tocsr(), brhs)
    t_2 = time.time() - t_0

    # Reshape the solution vector back into a 2D array
    phi = np.reshape(x, (ny, nx))

    print('Timing: set(A)=',t_1,' solve=',t_2,' seconds')

    return phi

# Main  calculation

1. Configude the domain and grid
1. Set some boundary conditions and a constraint ("source")
1. Initialize any obstacles
1. Solve! (Construct A matrix and do the linear algebra)
1. Compute any needed diagnostic quantities (velocity, pressure)
1. plot.

In [ ]:
# -----------------------------------------------------------
# RUN THE CALCULATION
# -----------------------------------------------------------
# Define background freestream velocity [m/s]
Ufree = 10

# Define domain size ([m] and grid spacing) 
Lx, Ly = 20, 10                  # lengths of X and Y domain
nx, ny = 201, 101                # number of points in computational mesh
#nx, ny = 1001, 501              # 1000x500 is a useful max on google: takes ~ 30 seconds

dx, dy = Lx/(nx-1), Ly/(ny-1)

x = np.linspace(0, Lx, nx)
y = np.linspace(0, Ly, ny)
X, Y = np.meshgrid(x, y)

# Set boundary freestream conditions
phi_left = 0.
phi_right = Ufree*Lx
phi_free = np.linspace(phi_left, phi_right, num=nx)
bdiv = np.zeros((ny,nx)) 


# Define an obstacle mask (multiple obsticles)
obs_list = []
obs_list.append(Obstacle.as_shape(X,Y,x0=  Lx/3,y0=  Ly/3,radius=Ly/6,shape='circle'))
obs_list.append(Obstacle.as_shape(X,Y,x0=  Lx/3,y0=2*Ly/3,radius=Ly/6,shape='square'))
obs_list.append(Obstacle.as_shape(X,Y,x0=2*Lx/3,y0=  Ly/3,radius=Ly/6,shape='diamond'))
obs_list.append(Obstacle.as_shape(X,Y,x0=  Lx/2,y0=  Ly/2,radius=Ly/8,shape='half-circle'))
obs_list.append(Obstacle.as_shape(X,Y,x0=3*Lx/4,y0=3*Ly/4,radius=Ly/8,shape='semi-circle'))

bmask = make_bmask(X, Y, obs_list)    # full obstable mask (for plotting)

# Solve potential flow problem
phi = solve_poisson_direct(dy, dx, phi_free, bmask, bdiv)

# Velocities from gradient (centred. One sideded at edges better)
v, u = np.gradient(phi, dy, dx)

# Pressure [Pa], from Bernoilli
p0 = 0.0       # background pressure [Pa]
rho = 1.0      # density [kg/m3]
pressure = p0 - 0.5*rho*(u**2 + v**2)   # allways negative to ambient

vspd = np.sqrt(u**2 + v**2) - Ufree

# PLOTTING - this takes a while ...
vstep = 3                   # don't plot every wind vector (else too many!)
fig, ax = plt.subplots(figsize=(12,7))
ax.set(aspect='equal')
plt.contourf(X, Y, -np.log(-pressure), levels=12,cmap='jet')  # Filled pressure 
plt.colorbar()
plt.contour(X, Y, phi,colors="blue", levels=13)     # potential lines
plt.streamplot(X, Y, u, v)
#plt.quiver(X[::vstep,::vstep], Y[::vstep,::vstep], u[::vstep,::vstep], v[::vstep,::vstep])
plt.contour(X, Y, vspd, colors="red")       # flow speed
plt.contour(X, Y, bmask, colors="black")
plt.show()